In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
import datetime

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [3]:
df = pd.read_csv('Hold/df2.csv')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays_3,pdays_4,pdays_5,previous_1,previous_2,previous_3,prop_previous_by_age,prop_previous_by_job,prop_previous_by_marital,prop_previous_by_education
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,0,0,1,0,0,1,0.133523,0.137736,0.155608,0.147031
1,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,0,0,1,0,0,1,0.133523,0.137736,0.155608,0.147031
2,56,housemaid,married,basic.4y,no,yes,no,telephone,may,fri,...,0,0,1,0,0,1,0.133523,0.137736,0.155608,0.147031
3,56,housemaid,married,basic.4y,no,no,no,telephone,jun,wed,...,0,0,1,0,0,1,0.133523,0.137736,0.155608,0.147031
4,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,0,0,1,0,0,1,0.133523,0.137736,0.155608,0.147031


In [5]:
cols = list(df.iloc[:,74:114].columns)

In [6]:
df1 = df.drop(cols, axis=1)

In [28]:

y = df1.y
x2 = df1.drop(['y', 'duration', 'emp.var.rate', 'euribor3m', 'previous_campaign_nonexistent', 'job', 'marital', 'education', 'contact', 'month', 'day_of_week','poutcome','default','housing','loan'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(x2, y, test_size = .3, random_state=42)

In [26]:
# Alternative set of features
# drop target variable y, and the following due to 
# high correlation: emp.var.rate, euribor3m, and previous_campaign_nonexistent

#x3 = df1.drop(['y', 'duration', 'emp.var.rate', 'euribor3m', 'previous_campaign_nonexistent', 'default', 'housing', 'loan',
#     'contact', 'month', 'day_of_week','poutcome'], axis=1)

IndentationError: unexpected indent (<ipython-input-26-1d58174bf6e9>, line 6)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(x3, y, test_size = .3, random_state=42)

X_train = pd.concat([X_train, y_train], axis=1)

a = X_train.groupby('age')['y'].sum()
b = X_train.groupby('age')['y'].count()
c = a/b
c = c.reset_index().rename(columns={'y': 'prop_y_by_age'})
X_train = X_train.merge(c, on='age')
X_test = X_test.merge(c, on='age')

a = X_train.groupby('job')['y'].sum()
b = X_train.groupby('job')['y'].count()
c = a/b
c = c.reset_index().rename(columns={'y': 'prop_y_by_job'})
X_train = X_train.merge(c, on='job')
X_test = X_test.merge(c, on='job')

a = X_train.groupby('marital')['y'].sum()
b = X_train.groupby('marital')['y'].count()
c = a/b
c = c.reset_index().rename(columns={'y': 'prop_y_by_marital'})
X_train = X_train.merge(c, on='marital')
X_test = X_test.merge(c, on='marital')

a = X_train.groupby('education')['y'].sum()
b = X_train.groupby('education')['y'].count()
c = a/b
c = c.reset_index().rename(columns={'y': 'prop_y_by_education'})
X_train = X_train.merge(c, on='education')
X_test = X_test.merge(c, on='education')

X_train=X_train.drop(['y', 'job','marital','education'], axis=1)

X_train.iloc[:,:4] = X_train.iloc[:,:4].astype(int)
X_train.iloc[:,7:59] = X_train.iloc[:,7:59].astype(int)

In [29]:
rfc = RandomForestClassifier(random_state=0)

In [30]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [31]:
rfc.feature_importances_

array([  1.07728860e-01,   7.19112406e-02,   2.27456921e-02,
         1.63406116e-02,   4.46913306e-02,   2.27682984e-02,
         8.56386571e-02,   1.09655030e-02,   7.83289391e-03,
         4.21423683e-03,   3.14899155e-03,   6.84808487e-03,
         3.86357669e-03,   5.16590808e-03,   7.04725115e-03,
         3.49248108e-03,   1.01531158e-02,   4.06327752e-03,
         1.44226799e-03,   7.79831256e-03,   1.02628573e-02,
         9.64491284e-03,   3.38816905e-04,   5.84146387e-03,
         4.06991615e-03,   8.15550711e-03,   1.29963163e-02,
         3.05483061e-04,   8.83192291e-03,   1.12593723e-02,
         5.21229571e-03,   7.64653473e-03,   8.51145342e-03,
         7.99207730e-08,   2.04761427e-02,   1.97096936e-03,
         1.87367164e-02,   1.29214805e-02,   2.55391117e-03,
         1.37006230e-02,   9.10767022e-03,   7.99603522e-03,
         3.45331191e-03,   4.10513007e-03,   1.09804575e-03,
         3.52799082e-03,   3.89312024e-03,   8.11432512e-03,
         4.93971634e-03,

In [32]:
X_train.columns

Index(['age', 'campaign', 'pdays', 'previous', 'cons.price.idx',
       'cons.conf.idx', 'nr.employed', 'entrepreneur', 'blue-collar',
       'services', 'management', 'unemployed', 'student', 'self-employed',
       'housemaid', 'admin.', 'technician', 'retired', 'unknown_profession',
       'divorced', 'married', 'single', 'unknown_marital_status', 'illiterate',
       'basic.6y', 'university.degree', 'basic.4y', 'basic.9y', 'high.school',
       'unknown_education', 'professional.course', 'yes_default',
       'unknown_default', 'no_default', 'yes_housing', 'unknown_housing',
       'no_housing', 'yes_loan', 'unknown_loan', 'no_loan', 'telephone',
       'cellular', 'oct', 'jul', 'jun', 'mar', 'nov', 'dec', 'apr', 'aug',
       'may', 'sep', 'mon', 'tue', 'fri', 'thu', 'wed',
       'previous_campaign_failure', 'previous_campaign_success',
       'prop_previous_by_age', 'prop_previous_by_job',
       'prop_previous_by_marital', 'prop_previous_by_education'],
      dtype='object')

In [33]:
importance = list(rfc.feature_importances_)
feature = list(X_train.columns)

feature_importance = pd.DataFrame({'Feature': feature, 'Importance': importance})

print('Feature importance determined from random forest')
print(feature_importance.sort_values('Importance', ascending=False))

Feature importance determined from random forest
                       Feature    Importance
59        prop_previous_by_age  1.082864e-01
0                          age  1.077289e-01
6                  nr.employed  8.563866e-02
1                     campaign  7.191124e-02
4               cons.price.idx  4.469133e-02
58   previous_campaign_success  4.014881e-02
60        prop_previous_by_job  3.363534e-02
62  prop_previous_by_education  2.972945e-02
5                cons.conf.idx  2.276830e-02
2                        pdays  2.274569e-02
34                 yes_housing  2.047614e-02
53                         tue  1.954170e-02
36                  no_housing  1.873672e-02
56                         wed  1.860155e-02
52                         mon  1.826747e-02
54                         fri  1.818366e-02
55                         thu  1.774233e-02
3                     previous  1.634061e-02
61    prop_previous_by_marital  1.490376e-02
39                     no_loan  1.370062e-02
26    

In [22]:
X_test = X_test.drop(['job', 'marital', 'education'],axis=1)

In [34]:
train_score = rfc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = rfc.score(X_test, y_test)
print("Test: {}".format(test_score))

Train: 0.9816863792445631
Test: 0.8890507404709881


In [35]:
# Running random search on RFC
print('Starting random search on RFC at: ', datetime.datetime.now())

param_grid = { 
              "n_estimators": np.arange(300, 400, 10),
              "max_features" : ['auto'],
              "min_samples_leaf": np.arange(7,13,1),
              'max_depth' : np.arange(25, 55, 5)
}

rfc = RandomForestClassifier(n_jobs=-1, oob_score= True, random_state = 0)
print('classifier substantiated')
    
CV_rfr = RandomizedSearchCV(estimator=rfc, param_distributions=param_grid, cv=3)
print('search substantiated')
    
print("Start random fit: ", datetime.datetime.now())
CV_rfr.fit(X_train, y_train)
print("End random fit: ", datetime.datetime.now())

print('Best Params: ', CV_rfr.best_params_)
bp = CV_rfr.best_params_ # best parameters

rfc = RandomForestClassifier(
n_estimators = bp['n_estimators'], 
max_features= bp['max_features'],
min_samples_leaf= bp['min_samples_leaf'], 
max_depth=bp['max_depth'])
    
print('Classifier with best parameters set')
     
print("Start rfr fit: ", datetime.datetime.now())
rfc.fit(X_train, y_train)
print("End rfr fit: ", datetime.datetime.now())

train_score = rfc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = rfc.score(X_test, y_test)
print("Test: {}".format(test_score))

print('Ending random search on RFR at: ', datetime.datetime.now())


Starting random search on RFC at:  2018-04-19 23:10:24.171250
classifier substantiated
search substantiated
Start random fit:  2018-04-19 23:10:24.172186
End random fit:  2018-04-19 23:12:16.810177
Best Params:  {'n_estimators': 330, 'min_samples_leaf': 8, 'max_features': 'auto', 'max_depth': 30}
Classifier with best parameters set
Start rfr fit:  2018-04-19 23:12:16.815271
End rfr fit:  2018-04-19 23:12:26.794812
Train: 0.9099580312857688
Test: 0.9007040543821316
Ending random search on RFR at:  2018-04-19 23:12:29.949358


In [37]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
train_score = lr.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = lr.score(X_test, y_test)
print("Test: {}".format(test_score))

Train: 0.8989282369671534
Test: 0.9015133122926277


In [38]:
print('Starting random search on RFC at: ', datetime.datetime.now())

param_grid = { 
              "C": [0.001,0.01,0.1,1,10,100]
}

lr = LogisticRegression(n_jobs=-1, random_state = 0)
print('classifier substantiated')
    
CV_lr = GridSearchCV(estimator=lr, param_grid=param_grid, cv=3)
print('search substantiated')
    
print("Start random fit: ", datetime.datetime.now())
CV_lr.fit(X_train, y_train)
print("End random fit: ", datetime.datetime.now())

print('Best Params: ', CV_lr.best_params_)
bp = CV_lr.best_params_ # best parameters

lr = LogisticRegression(
C = bp['C'],  
penalty = 'l2')
    
print('Classifier with best parameters set')
     
print("Start rfr fit: ", datetime.datetime.now())
lr.fit(X_train, y_train)
print("End rfr fit: ", datetime.datetime.now())

train_score = lr.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = lr.score(X_test, y_test)
print("Test: {}".format(test_score))

print('Ending random search on RFR at: ', datetime.datetime.now())

Starting random search on RFC at:  2018-04-19 23:12:50.683171
classifier substantiated
search substantiated
Start random fit:  2018-04-19 23:12:50.684944
End random fit:  2018-04-19 23:12:55.062084
Best Params:  {'C': 0.1}
Classifier with best parameters set
Start rfr fit:  2018-04-19 23:12:55.063033
End rfr fit:  2018-04-19 23:12:55.388998
Train: 0.8989976067427422
Test: 0.9016751638747269
Ending random search on RFR at:  2018-04-19 23:12:55.410525


In [39]:
mlpc = MLPClassifier()
mlpc.fit(X_train, y_train)
train_score = mlpc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = mlpc.score(X_test, y_test)
print("Test: {}".format(test_score))

Train: 0.8904651243453228
Test: 0.8910738852472283


In [40]:
# Running random search on MLPR
print('Starting MLPR random search at: ', datetime.datetime.now())

param_grid = { 
    "hidden_layer_sizes" : [1 ,2, 3],
    "alpha" : [.0001, .001, .01],
    "batch_size" : [1, 10, 50, 100, 250],
    "max_iter"  : np.arange(1250, 2250, 250)
            }

mlpc = MLPClassifier(random_state = 0) 

rs_mlpc = RandomizedSearchCV(estimator=mlpc, param_distributions=param_grid, cv= 3)

rs_mlpc.fit(X_train, y_train)
print('data fitted')
    
bp = rs_mlpc.best_params_ # best parameters
print(rs_mlpc.best_params_)

    
mlpc = MLPClassifier(random_state=0, hidden_layer_sizes = bp['hidden_layer_sizes'], 
alpha=bp['alpha'],
batch_size=bp['batch_size'], 
max_iter=bp['max_iter'])

print('regressor substantiated with best params')
    
mlpc.fit(X_train, y_train)
print('data fitted with best params')

train_score = mlpc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = mlpc.score(X_test, y_test)
print("Test: {}".format(test_score))

print('Ending MLPR random search at: ', datetime.datetime.now())


Starting MLPR random search at:  2018-04-19 23:12:59.180167
data fitted
{'max_iter': 1750, 'hidden_layer_sizes': 2, 'batch_size': 250, 'alpha': 0.001}
regressor substantiated with best params
data fitted with best params
Train: 0.8952169539731539
Test: 0.8999757222626851
Ending MLPR random search at:  2018-04-19 23:20:00.697450


In [41]:
knnc = KNeighborsClassifier()
knnc.fit(X_train, y_train)
train_score = knnc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = knnc.score(X_test, y_test)
print("Test: {}".format(test_score))

Train: 0.9133918351774132
Test: 0.8898599983814842


In [42]:
# Running random search on MLPR
print('Starting KNN random search at: ', datetime.datetime.now())

param_grid = { 
    'n_neighbors' : np.arange(2,10,2),
    'leaf_size' : np.arange(15,45,5)
            }

knnc = KNeighborsClassifier() 

rs_knnc = RandomizedSearchCV(estimator=knnc, param_distributions = param_grid, cv= 3)

rs_knnc.fit(X_train, y_train)
print('data fitted')
    
bp = rs_knnc.best_params_ # best parameters
print(rs_knnc.best_params_)


Starting KNN random search at:  2018-04-19 23:20:18.996934
data fitted
{'n_neighbors': 8, 'leaf_size': 40}


In [43]:
knnc = KNeighborsClassifier(n_neighbors = bp['n_neighbors'], 
leaf_size=bp['leaf_size'])

print('regressor substantiated with best params')
    
knnc.fit(X_train, y_train)
print('data fitted with best params')

train_score = knnc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = knnc.score(X_test, y_test)
print("Test: {}".format(test_score))

print('Ending MLPR random search at: ', datetime.datetime.now())


regressor substantiated with best params
data fitted with best params
Train: 0.9057264749748535
Test: 0.8954438779639071
Ending MLPR random search at:  2018-04-19 23:25:31.252976


In [44]:
svc = SVC()
svc.fit(X_train, y_train)
train_score = svc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = svc.score(X_test, y_test)
print("Test: {}".format(test_score))

Train: 0.9046859283410218
Test: 0.8998138706805859


In [45]:
# Running random search on 
print('Starting SVC random search at: ', datetime.datetime.now())

param_grid = { 
    'C' : [0.001, 0.01, 0.1, 1, 10],
    'gamma' : [0.001, 0.01, 0.1, 1]
            }


svc = SVC()

rs_svc = RandomizedSearchCV(estimator=svc, param_distributions = param_grid, cv= 3)

rs_svc.fit(X_train, y_train)
print('data fitted')
    
bp = rs_svc.best_params_ # best parameters
print(rs_svc.best_params_)

svc = SVC(C = bp['C'], 
gamma=bp['gamma'])

print('regressor substantiated with best params')
    
svc.fit(X_train, y_train)
print('data fitted with best params')

train_score = svc.score(X_train, y_train)
print("Train: {}".format(train_score))

test_score = svc.score(X_test, y_test)
print("Test: {}".format(test_score))

print('Ending MLPR random search at: ', datetime.datetime.now())



Starting SVC random search at:  2018-04-19 23:28:15.714341
data fitted
{'gamma': 0.001, 'C': 0.1}
regressor substantiated with best params
data fitted with best params
Train: 0.8990322916305366
Test: 0.900623128591082
Ending MLPR random search at:  2018-04-20 00:00:57.777211
